# Human Caption

In [1]:
import pickle

# gt captions
human_captions_path="./Cap3D_human_Objaverse.pkl"

with open(human_captions_path, 'rb') as f:
    human_captions = pickle.load(f)     #dict

In [16]:
fewshots_uid=["00ac0c1e6c0d402dbd6c92e71f23d7f8",
              #"00c86cb620a743cc81d9cb87ad4d8696",
              "00e7ff0b7596430bbecaf9456cb43ff6",
              #"00fbc8e52f834ba78b25ffb6962b98e8"
              ]
for uid in fewshots_uid:
    print(f"{uid}:{human_captions[uid]}\n")

KeyError: '00ac0c1e6c0d402dbd6c92e71f23d7f8'

# Qwen3-vl-plus

In [2]:
import os
import pickle   

def diffurank_select(obj_path):
    diffu_path = os.path.join(obj_path, "diffurank_scores.pkl")
    with open(diffu_path, 'rb') as f:
        diffu_scores = pickle.load(f)     # numpy array

    #print(diffu_scores)
    
    # 将索引和分数配对，按分数排序（升序）
    indexed = list(enumerate(diffu_scores))
    indexed.sort(key=lambda x: x[1])
    # 取分数最低的6个
    lowest_six = indexed[:6]
    # 提取索引（如果需要反转顺序，使用 [::-1]）
    indices = [idx for idx, val in lowest_six][::-1]   # 保持与排序后相同的顺序
    # 如果想按分数从低到高对应的索引顺序，可直接返回 indices
    # 如果想按原数组顺序（即索引升序），可以再排序 indices.sort()
    return indices

print(diffurank_select("./data/00ac0c1e6c0d402dbd6c92e71f23d7f8"))

[0.3094799  0.30376685 0.29219562 0.280645   0.29946023 0.28154358
 0.2857333  0.27162117 0.3115896  0.31115136 0.28345588 0.27608347
 0.30614647 0.2969632  0.29660958 0.29761234 0.3093227  0.29578447
 0.27920803 0.3149979 ]
[10, 5, 3, 18, 11, 7]


In [5]:
prompt_text="以上是一个3D物体的6张渲染图。人工描述的标注为 a sofa with four legs ,请你用英文，用一到两句简介的话写出得到这个标注的思考过程，抓住关键点"

In [11]:
prompt_text1="以上是一个3D物体的6张渲染图。人工描述的标注为 A futuristic wrist-mounted device featuring a wooden-textured control panel with digital displays, indicator lights, and buttons 。请你假装将它误认为一个hairpin，用英文，用一到两句简洁的话写出得到这个标注的思考过程"

In [13]:
prompt_text2="以上是一个3D物体的6张渲染图。人工描述的标注为 A futuristic wrist-mounted device featuring a wooden-textured control panel with digital displays, indicator lights, and buttons 。假如你将它误认为一个hairpin，你的思考过程是：its curved, symmetrical metal band resembles a classic double-pronged hair clip, and the central “panel” looked like a decorative embellishment.请你用英文，用一到两句简洁的话说出你的自我反思，也就是做出错误描述的可能原因和a new, concise, high level plan that aims to mitigate the same failure."

In [14]:
import os
import dashscope

# 以下为北京地域base_url，若使用弗吉尼亚地域模型，需要将base_url换成 https://dashscope-us.aliyuncs.com/api/v1
# 若使用新加坡地域的模型，需将base_url替换为：https://dashscope-intl.aliyuncs.com/api/v1
dashscope.base_http_api_url = "https://dashscope.aliyuncs.com/api/v1"


obj_uid="00e7ff0b7596430bbecaf9456cb43ff6"
data_dir="./data"

obj_path=os.path.join(data_dir,obj_uid)

idx_list=diffurank_select(obj_path)

local_path1 = os.path.join(obj_path,f"{idx_list[0]:05}.png")
local_path2 = os.path.join(obj_path,f"{idx_list[1]:05}.png")
local_path3 = os.path.join(obj_path,f"{idx_list[2]:05}.png")
local_path4 = os.path.join(obj_path,f"{idx_list[3]:05}.png")
local_path5 = os.path.join(obj_path,f"{idx_list[4]:05}.png")
local_path6 = os.path.join(obj_path,f"{idx_list[5]:05}.png")

image_path1 = f"file://{local_path1}"
image_path2 = f"file://{local_path2}"
image_path3 = f"file://{local_path3}"
image_path4 = f"file://{local_path4}"
image_path5 = f"file://{local_path5}"
image_path6 = f"file://{local_path6}"

messages = [
    {
        "role": "user",
        "content": [
            {"image": image_path1},
            {"image": image_path2},
            {"image": image_path3},
            {"image": image_path4},
            {"image": image_path5},
            {"image": image_path6},
            {"text": prompt_text2}
        ]
    }
]

response = dashscope.MultiModalConversation.call(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    # 各地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
    #api_key=os.getenv('DASHSCOPE_API_KEY'),
    api_key="sk-e7d194c626aa4fcf90aeb0bbdfe59c6b",
    model='qwen3-vl-plus', # 此处以qwen3.5-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/models
    messages=messages
)

print(response.output.choices[0].message.content[0]["text"])

[0.32248756 0.3139494  0.34247452 0.32250738 0.31663415 0.32238635
 0.31792325 0.31256446 0.28798184 0.3168126  0.32756263 0.3471156
 0.29463798 0.30853695 0.31920066 0.32287425 0.30137026 0.32686064
 0.33777407 0.3214774 ]
My misidentification likely stemmed from overemphasizing the object’s curved, symmetrical form while underweighting functional cues (e.g., digital displays, buttons, wrist-scale context); to prevent recurrence, I will prioritize *functional affordances* and *scale-relative features* over superficial shape analogies when interpreting 3D renders.
